

# Conclusions from EDA

### Data Integrity & Completeness

* A small percentage of games (14.92%) are missing category information, which could affect category-based analyses. However, we decided not to pursue a content-based recommendation approach due to multiple issues in our code that would render such methods ineffective. Among the games that do have category data, 29.60% are assigned to multiple categories, making it difficult to pinpoint what uniquely defines a game within a specific category. Additionally, the dataset contains very limited content-based features. When analyzing rating distributions per category, they appeared very similar, making it challenging to extract meaningful insights.

* A significant portion of games (30.07%) have no reviews. Since our approach does not involve content-based recommendations, we will exclude these games from further analysis.



### Game Ratings and Popularity

* There is a trend showing that games with more ratings tend to receive slightly higher average ratings. This suggests that more popular games are generally better received.

* The overall distribution of game ratings skews toward the higher end. This aligns with common social tendencies where ratings below 6 are often perceived as "bad," leading to an inflated average. This bias should be considered in our interpretation of rating data.


### Reviewer Behavior

* A small number of reviewers contribute a large proportion of the reviews (e.g., the top reviewer has significantly more reviews than the average user).

* Review activity has been consistent over the years, with some peaks in recent years. Interestingly, the average rating has increased over time, which could be due to a shift in user behavior—possibly fewer people reviewing in recent years, and those who do are primarily rating games they enjoyed.

* We categorized users into "Frequent Reviewers" (≥18 reviews, determined later in the project using a ping-pong algorithm) and "Casual Reviewers" (<18 reviews). These groups differ in their rating behavior: frequent reviewers tend to be more critical, especially in earlier years, though the average review length between the two groups is quite similar.

* Frequent reviewers (17.7% of users) account for a large share of total ratings (80.6%), highlighting their outsized influence on the dataset.

* Most reviews are relatively short, which could affect the performance and reliability of interpretability-focused models using review text.


### Review Content

* There is no strong visual correlation between review length and the numerical rating. However, analyzing this relationship helped us identify at least one spam review.



### Rating Consistency

* Games with a higher number of ratings tend to have lower standard deviations in those ratings. This suggests that with more reviews, there is a stronger consensus on a game's quality.
* Most people rate “in the middle,” but there’s a sizable minority who almost always give glowing 10s, and a smaller group who almost always give very low scores.

### Problems to consider
* The number of review are really low in comparison to the number of user*games without any cleaning we have 196814241 missing values with a fill rate of 0.08%




# Cleaning

* Removing the spam reviews
* Removing duplicates
* Creating a sparse matrix to optimize the storage because of the the very low fill rate

### Finding the Optimal Threshold to Subset the Data

* We used a "ping-pong" algorithm where we iteratively removed games and users based on minimum review counts—alternating between the two—until no further removals were needed. For each combination of minimum game and user review thresholds, we calculated a custom harmonic mean that combined: Data retention (the proportion of ratings retained), Normalized density (the fill rate of the matrix), Inverted normalized KL-divergence (to favor uniformity in rating distributions)
* Based on this approach, the optimal thresholds were: Minimum user ratings: 14 / Minimum game ratings: 18
* Final dataset statistics:
Number of users: 1,753 / Number of games: 1,824 / Number of ratings: 99,928 / Average number of ratings per user: 57.00 / Average number of ratings per game: 54.79
* We then applied these thresholds on our matrix

# The models
 
To test our methods, especially when we didn't have a lot of ratings (which we call "realistic sparsity"), we first set up a special way to split our data for training and testing. Instead of completely hiding all ratings for some users or games, we randomly hid individual ratings. We also added settings to control how many ratings we removed for each user or game.

Next, we figured out three simple baseline scores to compare against:
* **Global Mean:** This was just the average of all the ratings we could see.
* **User Mean:** For each user, this was their average rating across all the games they had rated.
* **Game Mean:** For each game, this was its average rating from all the users who had rated it.

After that, we used a tool called the "Surprise" library to try out several common models with our dataset. We started with its basic `BaselineOnly` model, and then tested Non-Negative Matrix Factorization (NMF), KNNBasic, KNNWithMeans, and SVD, all using their standard default settings.

Finally, for one more comparison, we used a k-nearest-neighbors regressor (from the "scikit-learn" library). 
With our models(trained with our custom train/test split), we first subtracted each user's average rating from their actual ratings. We did this because not everyone rates the same way – some people generally give higher scores, and others give lower ones. By taking away each user's average first, we remove that personal scoring habit. This helps the model focus on which items a user likes more or less than their own usual, instead of just whether they are a "tough" or "easy" rater.
We also wanted to see with Knnbasics how much of a difference it makes if you dont substract the mean.

### With the **Default** parameters we discoverd a few things from the results:
- The most striking finding is that BaselineOnly a relatively simple model that just captures global average, user bias, and game bias - outperforms more sophisticated algorithms. With the best RMSE (1.7772) and MAE (1.3632), this suggests that much of the rating behavior in your dataset can be explained by systematic biases rather than complex user-game interactions.
- While BaselineOnly wins on error metrics (RMSE, MAE), the sklearn KNN model has the best R² score (0.3150). This means: BaselineOnly minimizes absolute prediction errors and KNN better explains the variance in user ratings
- SVD performs quite well (3.0 avg rank), while NMF performs poorly (8.33 avg rank). This indicates that:
- SVD performs quite well (3.0 avg rank), while NMF performs poorly (8.33 avg rank). This indicates that:
    - The constraints of non-negativity in NMF may be limiting its ability to model your data . It could be also that we didnt optmize our NMF parameters
    - SVD's ability to capture both positive and negative latent factors is beneficial for your dataset
- The results also displayed that substracting the mean really helped with the performences of the knn as KnnWithMean out performs Knnbasic in all 3 metrics


# Model errors

- All models find it hardest to predict for items with very few ratings.
- here seems to be a "sweet spot" in the "Low" to "Medium" popularity range where items have enough data for models to learn from but haven't yet attracted the highly diverse (and sometimes harder to predict) rating patterns of extremely popular items.
- Popularity Isn't Everything: Simply having more ratings (Very High popularity) doesn't guarantee the lowest prediction error, likely due to the increased diversity of opinions and rating behaviors associated with such items.
- KNN's Dependency on Sufficient Data: The neighborhood-based approach of KNN is powerful but requires a certain density of data to form meaningful, stable neighborhoods.
- More Data Helps (Especially for Complex Models): As users provide more ratings ("Very High" activity), all models improve, with the more complex models (KNN, NMF, SVD) benefiting significantly.
- Baseline is a Strong Contender for Sparse Users

The model tend to do overestimation for low true ratings:
- The biggest errors occur when instances where a user's rating for a specific gale goes strongly against their own typical behavior or against the item's general perception, or against what their "similar" users would do.
- Models often smooth things out: Recommendation models, especially collaborative filtering ones, tend to predict ratings that are somewhat "safer" or closer to averages (user average, item average, or an average learned from neighbors/factors). They struggle with extreme, idiosyncratic opinions unless there's very strong evidence in the training data for that specific user-item interaction.
-  Users tend to rate items they engage with. While not always true, there might be more data points reflecting positive or neutral interactions than extremely negative ones for items they bothered to rate, especially for users who generally enjoy the hobby (and thus might have higher average ratings).
- Models are often optimized to minimize overall error (like RMSE). A few very large errors on outlier opinions might be "accepted" by the model if it means getting many other predictions more accurate by a smaller margin.

But also: 
- All models struggle when an individual user has a strong positive preference for an item that is, on average, not well-rated by the community or is perhaps more obscure/niche. The models are pulled towards the lower item average or lack strong collaborative signals to predict such a high rating.
- When a user gives an exceptionally high rating to an item, especially if that item isn't universally acclaimed or if the user themself isn't always a high-rater, models can be conservative and underpredict. They might regress towards the user's mean or the item's mean.

Model-Specific Observations (and how they relate to their nature):


- BaselineOnly:

    Observation: Consistently predicts ratings around 4.3 - 5.1 for items users rated 10.0. These predicted values are often closer to a blend of the user_mean_rating and the (often low) item_mean_rating.
        Conclusion: This is expected. BaselineOnly simply uses overall average + user bias + item bias. If a user generally rates low/medium (e.g., rugal at 4.26) and an item is also rated low/medium on average (e.g., Fantasy at 4.5), even if this user gives this item a 10, the baseline prediction will be pulled down by both these averages. It cannot capture this specific, strong positive interaction.

- KNNWithMeans:
 
    Observation: Can have very large underestimations (e.g., greuh for Battletech, true 10.0, pred 2.75; scorion for Z first impact..., true 7.7, pred 0.94).
        Conclusion: This suggests that for these user-item pairs, the user's "neighbors" (users with similar overall rating patterns) either did not rate these specific items highly, rated them poorly, or didn't rate them at all. If the neighborhood doesn't provide strong positive evidence for that specific item, KNN will struggle, even if the user themselves loved it. The error can be particularly large if the neighbors actively disliked it. The low item_n_ratings for some of these also means fewer neighbors might have rated the item at all.

- NMF and SVD (Matrix Factorization):

    Observation: Still significantly underestimate (e.g., pééétrick for Brainstorm!, true 10.0, pred 4.89 for NMF; Kootoh for Lunch Money, true 10.0, pred 5.13 for SVD). The magnitude of underestimation can still be large.
        Conclusion: These models try to learn latent factors representing user preferences and item characteristics. If a user's strong liking for a particular (often niche or lower-average-rated) item is a very unique aspect of their taste that isn't well-represented by the learned latent factors (or if the item's latent factors don't strongly signal "high appeal" to this user's factor profile), the models will underpredict. They are essentially saying, "This user-item interaction doesn't strongly fit the general patterns of preference I've learned from the whole dataset

### Ideas we tried but that didnt work 

- fill the nan value with some sort of algorithm or disturbition to find the patterns in our data but with a fill rate so low initially this wasnt a good idea